In [12]:
%%shell
jupyter nbconvert --to html /content/P1.ipynb

[NbConvertApp] Converting notebook /content/P1.ipynb to html
[NbConvertApp] Writing 354920 bytes to /content/P1.html


# Data Science Portfolio - Part I (30 marks)

In this question you will write Python code for processing, analyzing and understanding the social network **Reddit** (www.reddit.com). Reddit is a platform that allows users to upload posts and comment on them, and is divided in _subreddits_, often covering specific themes or areas of interest (for example, [world news](https://www.reddit.com/r/worldnews/), [ukpolitics](https://www.reddit.com/r/ukpolitics/) or [nintendo](https://www.reddit.com/r/nintendo)). You are provided with a subset of Reddit with posts from Covid-related subreddits (e.g., _CoronavirusUK_ or _NoNewNormal_), as well as randomly selected subreddits (e.g., _donaldtrump_ or _razer_).

The `csv` dataset you are provided contains one row per post, and has information about three entities: **posts**, **users** and **subreddits**. The column names are self-explanatory: columns starting with the prefix `user_` describe users, those starting with the prefix `subr_` describe subreddits, the `subreddit` column is the subreddit name, and the rest of the columns are post attributes (`author`, `posted_at`, `title` and post text - the `selftext` column-, number of comments - `num_comments`, `score`, etc.).

In this exercise, you are asked to perform a number of operations to gain insights from the data.

In [1]:
# C21102236
# suggested imports
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()
from ast import literal_eval
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from urllib import request
import pandas as pd
module_url = f"https://raw.githubusercontent.com/luisespinosaanke/cmt309-portfolio/master/data_portfolio_21.csv"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))


df = pd.read_csv('data_portfolio_21.csv')
# this fills empty cells with empty strings
df = df.fillna('')

Fetching https://raw.githubusercontent.com/luisespinosaanke/cmt309-portfolio/master/data_portfolio_21.csv


In [3]:
df.head(10)

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599
5,-Howitzer-,2020-11-17 16:33:29,30,26,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Here Is The Evidence – Crowdsourcing Evidence ...,0,0.67,4661,2012-11-09,-0.658599
6,-Howitzer-,2020-08-19 17:04:16,45,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,PJWs,0,1.00,4661,2012-11-09,-0.658599
7,-Howitzer-,2020-09-04 13:38:53,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,CAN'T CUCK THE TUCK,0,1.00,4661,2012-11-09,-0.658599
8,-Howitzer-,2020-09-03 20:16:52,6,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Portland Antifa is Being Housed By City Democr...,0,1.00,4661,2012-11-09,-0.658599
9,-Howitzer-,2020-06-30 22:13:30,7,2,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"Citing State of Affairs in US, Schiff Jokes 'W...",0,1.00,4661,2012-11-09,-0.658599


## P1.1 - Text data processing (10 marks)

### P1.1.1 - Faved by as lists (3 marks)

The column `subr_faved_by` contains an array of values (names of redditors who added the subreddit to which the current post was submitted), but unfortunately they are in text format, and you would not be able to process them properly without converting them to a suitable python type. You must convert these string values to Python lists, going from

```python
'["user1", "user2" ... ]'
```

to

```python
["user1", "user2" ... ]
```

**What to implement:** Implement a function `transform_faves(df)` which takes as input the original dataframe and returns the same dataframe, but with one additional column called `subr_faved_by_as_list`, where you have the same information as in `subr_faved_by`, but as a python list instead of a string.

In [4]:
def transform_faves(df):
    """
    Function creates a new column "subr_faved_by_as_list" from the column "subr_faved_by" and inserts it into an original dataframe

    Parameters
    ----------
    df : Pandas DATAFRAME
        dataframe that includes, but not limited to a column "subr_faved_by" in a text format

    Returns
    -------
    df : Pandas DATAFRAME
        original dataframe with an inserted column "subr_faved_by_as_list" in a list format

    """
    df["subr_faved_by_as_list"] = "" # create a new column "subr_faved_by_as_list" in an original dataframe
    for n in range(len(df.subr_faved_by)): # iterate through "subr_faved_by" column
      x = re.sub(",|'", "", df.subr_faved_by[n]) # replace "," and "'" with "" in a captured value in a "subr_faved_by" column and assign a renewed string to a variable x
      x = x[1:-1] # remove "[" from the start and "]" from the end of the string x
      mylist = list(x.split(" ")) # reformat string x into a list format and assign it to mylist
      df.subr_faved_by_as_list[n] = mylist # let list mylist to be a value in a captured column "subr_faved_by_as_list"
    return df
df = transform_faves(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### P1.1.2 - Merge titles and text bodies (4 marks)

All Reddit posts need to have a title, but a text body is optional. However, we want to be able to access all free text information for each post without having to look at two columns every time.

**What to implement**: A function `concat(df)` that will take as input the original dataframe and will return it with an additional column called `full_text`, which will concatenate `title` and `selftext` columns, but with the following restrictions:

- 1) Wrap the title between `<title>` and `</title>` tags.
- 2) Add a new line (`\n`) between title and selftext, but only in cases where you have both values (see instruction 4).
- 3) Wrap the selftext between `<selftext>` and `</selftext>`.
- 4) You **must not** include the tags in points (1) or (3) if the values for these columns is missing. We will consider a missing value either an empty value (empty string) or a string of only one character (e.g., an emoji). Also, the value of a `full_text` column must not end in the new line character.

In [5]:
def concat(df):
    """
    Function creates an additional column called full_text, which concatenates title and selftext columns and inserts it into an original dataframe

    Parameters
    ----------
    df : Pandas DATAFRAME
        dataframe that includes, but not limited to columns "title" and "selftext"

    Returns
    -------
    df : Pandas DATAFRAME
        original dataframe with included "full_text" column

    """
    count = 0 # count for a while loop
    df["full_text"] = "" # creates another column in the given dataframe
    mylist = list() # creates a list mylist
    while count < len(df.title): # iterate through "title" column
      toAdd = "" # create an empty string toAdd
      title = df.title[count] # let a variable title be the captured value in a column "title"
      selftext = df.selftext[count] # let a variable selftext be the captured value in a column "selftext"
      if len(title) > 1: # check whether a variable title is either an empty value (empty string) or a string of only one character (e.g., an emoji)
        toAdd = "<title>" + title + "</title>" # wrap the variable title with title tags
        if len(selftext) > 1: # check whether a variable selftext is either an empty value (empty string) or a string of only one character (e.g., an emoji)
          toAdd = toAdd + "\n" + "<selftext>" + selftext + "</selftext>" # wrap the variable selftext with selftext tags and add "\n" to separate it from the title
      else: # if a variable title is either an empty value (empty string) or a string of only one character (e.g., an emoji)
        if len(selftext) > 1: # check whether a variable selftext is either an empty value (empty string) or a string of only one character (e.g., an emoji)
          toAdd = "<selftext>" + selftext + "</selftext>" # wrap the variable selftext with selftext tags
      df["full_text"][count] = toAdd # add a resulting string to "full_text" column
      count += 1 # add 1 to a count variable to continue an iteration process
    return df
df = concat(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### P1.1.3 - Enrich posts (3 marks)

We would like to augment our text data with linguistic information. To this end, we will _tokenize_, apply _part-of-speech tagging_, and then we will _lower case_ all the posts.

**What to implement**: A function `enrich_posts(df)` that will take as input the original dataframe and will return it with **two** additional columns: `enriched_title` and `enriched_selftext`. These columns will contain tokenized, pos-tagged and lower cased versions of the original text. **You must implement them in this order**, because the pos tagger uses casing information.

In [6]:
def enrich_posts(df):
    """
    Function adds two additional columns "enriched_title" and "enriched_selftext" to an original dataframe, which contain tokenized, pos-tagged and lower cased versions of the original "title" and "selftext" columns.

    Parameters
    ----------
    df : Pandas DATAFRAME
        dataframe that includes, but not limited to columns "title" and "selftext"

    Returns
    -------
    df : Pandas DATAFRAME
        original dataframe with included "enriched_title" and "enriched_selftext" columns

    """
    count = 0 # count for a while loop
    mylist = list() # creates a list mylist
    mytuple = tuple() # creates a tuple mytuple
    df["enriched_title"] = "" # creates another column in the given dataframe
    df["enriched_selftext"] = "" # creates another column in the given dataframe
    while count < len(df.title): # iterate through "title" column
      tokenized = word_tokenize(df.title[count]) # let a variable tokenized be a tokenized version of a captured value in a "title" column
      tagged = pos_tag(tokenized) # let a variable tagged be a tagged version of a variable tokenized
      for i in range(len(tagged)): # iterate through a variable tagged
        mytuple = (tagged[i][0].lower(), tagged[i][1]) # let a variable mytuple contain a lower cased string of the variable tagged and its tag 
        mylist.append(mytuple) # add a tuple mytuple to the list mylist
      df["enriched_title"][count] = mylist # add a resulting list to "enriched_title" column
      mylist = [] # empty a list mylist
      count += 1 # add 1 to a count variable to continue an iteration process
    mylist = [] # empty a list mylist
    count = 0 # let a variable count be 0 to be able to start a while loop from the beginning 
    while count < len(df.selftext): # iterate through "selftext" column
      tokenized = word_tokenize(df.selftext[count]) # let a variable tokenized be a tokenized version of a captured value in a "selftext" column
      tagged = pos_tag(tokenized) # let a variable tagged be a tagged version of a variable tokenized
      for i in range(len(tagged)): # iterate through a variable tagged
        mytuple = (tagged[i][0].lower(), tagged[i][1]) # let a variable mytuple contain a lower cased string of the variable tagged and its tag
        mylist.append(mytuple) # add a tuple mytuple to the list mylist
      df["enriched_selftext"][count] = mylist  # add a resulting list to "enriched_selftext" column
      mylist = [] # empty a list mylist
      count +=1 # add 1 to a count variable to continue an iteration process
    return df
df = enrich_posts(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df.head()

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,...,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list,full_text,enriched_title,enriched_selftext
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>BREAKING: Trump to begin hiding in mail...,"[(breaking, NN), (:, :), (trump, NN), (to, TO)...",[]
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Joe Biden's America</title>,"[(joe, NNP), (biden, NNP), ('s, POS), (america...",[]
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>4 more years and we can erase his legac...,"[(4, CD), (more, JJR), (years, NNS), (and, CC)...",[]
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Revelation 9:6 [Transhumanism: The New ...,"[(revelation, NN), (9:6, CD), ([, JJ), (transh...",[]
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...","<title>LOOK HERE, FAT</title>","[(look, NNP), (here, NNP), (,, ,), (fat, NNP)]",[]


## P1.2 - Answering questions with pandas (12 marks)

In this question, your task is to use pandas to answer questions about the data.

### P1.2.1 - Users with best scores (3 marks)

- Find the users with the highest aggregate scores (over all their posts) for the whole dataset. You should restrict your results to only those whose aggregated score is above 10,000 points, in descending order. Your code should generate a dictionary of the form `{author:aggregated_scores ... }`.

In [8]:
x = {} # let x be an empty dictionary
count = 0 # count for a while loop
while count < len(df.author): # iterate through "author" column
  if count == 0: # if it's a first iteration
    author = df.author[count] # let a variable author be a captured value in a "author" column
    score = df.score[count] # let a variable score be a captured value in a "score" column
    count += 1 # add 1 to a variable count to continue an iteration process
  else: # if it's not a first iteration
    if author == df.author[count]: # if a captured value in a "author" column matches an author variable
      score += df.score[count] # add its score to a score variable
    else: # if a captured value in a "author" column doesn't match an author variable
      if score > 10000: # if a score variable is larger than 10,000
        x[author] = score # add its key and value to a dictionary x
      author = df.author[count] # let a variable author be a captured value in a "author" column
      score = df.score[count] # let a variable score be a captured value in a "score" column
    count += 1 # add 1 to a variable count to continue an iteration process
count = 0 # let a variable count be 0 to be able to start a while loop from the beginning 
x_sorted_keys = sorted(x, key=x.get, reverse=True) # let a variable x_sorted_keys be sorted by keys of a variable x
x_sorted = {} # create an empty dictionary x_sorted
while count < len(x_sorted_keys): # iterate through a variable x_sorted_keys
  x_sorted[x_sorted_keys[count]] = x[x_sorted_keys[count]] # let an author's name be a key and an author's score be a value and add it to the dictionary x_sorted
  count += 1 # add 1 to a variable count to continue an iteration process
print(x_sorted) # print a dictionary of the users with the highest aggregate scores (over all their posts) for the whole dataset, whose aggregated score is above 10,000 points, in descending order.

{'DaFunkJunkie': 250375, 'None': 218846, 'SUPERGUESSOUS': 211611, 'jigsawmap': 210784, 'chrisdh79': 143538, 'hildebrand_rarity': 122464, 'iSlingShlong': 118595, 'hilltopye': 81245, 'tefunka': 79560, 'OldFashionedJizz': 64398, 'JLBesq1981': 58235, 'rspix000': 57107, 'Wagamaga': 47989, 'stem12345679': 47455, 'TheJeck': 26058, 'TheGamerDanYT': 25357, 'TrumpSharted': 21154, 'NotsoPG': 18518, 'SonictheManhog': 18116, 'BlanketMage': 13677, 'NewAltWhoThis': 12771, 'kevinmrr': 11900, 'Dajakesta0624': 11613, 'apocalypticalley': 10382}


### P1.2.2 - Awarded posts (3 marks)

Find the number of posts that have received at least one award. Your query should return only one value.

In [9]:
count = 0 # count for a while loop
result = 0 # let a variable result be 0
while count < len(df.total_awards_received): # iterate through a "total_awards_received" column
  if df.total_awards_received[count] > 0: # if a captured value in a "total_awards_received" column is greater than 0
    result += 1 # add 1 to a variable result
  count += 1 # add 1 to a variable count to continue an iteration process
print(result) # print the number of posts that have received at least one award

119


### P1.2.3 Find Covid (3 marks)

Find the name and description of all subreddits where the name starts with `Covid` or `Corona` and the description contains `covid` or `Covid` anywhere. Your code should generate a dictionary of the form#

```python
  {'Coronavirus':'Place to discuss all things COVID-related',
  ...
  }
```

In [10]:
count = 0 # count for a while loop
regexSub = re.compile("^Covid|^Corona") # let a variable regexSub be a regex for subreddits where the name starts with Covid or Corona
regexDesc = re.compile("Covid|covid") # let a variable regexDesc be a regex for descriptions that contain covid or Covid anywhere
d = {} # let d be an empty dictionary
while count < len(df.subreddit): # iterate through a "subreddit" column
  if re.search(regexSub, df.subreddit[count]) and re.search(regexDesc, df.subr_description[count]): # if a captured value in "subreddit" column matches regexSub and regexDesc regexs
    d[df.subreddit[count]] = df.subr_description[count] # add it to the dictionary d
  count += 1 # add 1 to a variable count to continue an iteration process
print(d) # print a resulting dictionary

{'CoronavirusCA': 'Tracking the Coronavirus/Covid-19 outbreak in California'}


### P1.2.4 - Redditors that favorite the most

Find the users that have favorited the largest number of subreddits. You must produce a pandas dataframe with **two** columns, with the following format:

```python
     redditor	    numb_favs
0	user1           7
1	user2           6
2	user3	       5
3	user4           4
...
```

where the first column is a Redditor username and the second column is the number of distinct subreddits he/she has favorited.

In [11]:
count = 0 # count for a while loop
countInt = 0 # internal count for a while loop
s = list() # let s be a list
redd = list() # let redd be a list
users = list() # lets users be a list
d = {} # let d be a dictionary
while count < len(df.subreddit): # iterate through a "subreddit" column
  if df.subreddit[count] not in s: # if a captured value in a "subreddit" column is not in the list s
    s.append(df.subreddit[count]) # add a captured value in a "subreddit" column to a list s
  count += 1 # add 1 to a variable count to continue an iteration process
count = 0 # let a variable count be 0 to be able to start a while loop from the beginning
while count < len(df.subr_faved_by_as_list): # iterate through a "subr_faved_by_as_list" column
  if count == 0: # if it's a first iteration
    redd.append(df.subr_faved_by_as_list[count]) # add a captured value in a "subr_faved_by_as_list" column to a list redd
    countInt += 1 # add 1 to a variable countInt to continue an iteration process
  else: # if it's not a first iteration
    if df.subreddit[count] == s[countInt]: # if a captured value in a "subreddit" column equals a captured value in a list s
      redd.append(df.subr_faved_by_as_list[count]) # add a captured value in a "subr_faved_by_as_list" column to a list redd
      countInt += 1 # add 1 to a variable countInt to continue an iteration process
  count += 1 # add 1 to a variable count to continue an iteration process
  if countInt == len(s): # if a variable countInt equals a length of a list s
    break # stop the iteration process
for n in range(len(s)): # iterate through list s
  for i in range(len(redd[n])): # iterate through a captured list within a list redd
    users.append(redd[n][i]) # add a captured value within a list of a list redd to a list users
for n in users: # iterate through a list users
  d[n] = users.count(n) # let the captured value from the list users be a key in a dictionary d and its number of occurrences be its value 
sorted_keys = sorted(d, key=d.get, reverse = True) # let sorted_keys be a sorted list of redditors by their number of favourites in a descending order
d1 = {} # let d1 be an empty dictionary
for n in sorted_keys: # iterate through a list sorted_keys
    d1[n] = d[n] # let the captured value from the list sorted_keys be a key in a dictionary d1 and its number of favourites be its value
favUsers = pd.DataFrame(d1.items(), columns=['redditor', 'numb_favs']) # turn a dictionary d1 into a dataframe and assign labels to the columns
print(favUsers) # print a pandas dataframe with two columns, where the first column is a Redditor username and the second column is the number of distinct subreddits he/she has favorited

                  redditor  numb_favs
0           magnusthered15          7
1                 ry_ta506          6
2                KarmaFury          6
3     FriendlyVegetable420          6
4              OmniusQubus          6
...                    ...        ...
1594   certifiedloverboy69          1
1595            diveonfire          1
1596         mouthofreason          1
1597               Alexify          1
1598        lilstinky[***]          1

[1599 rows x 2 columns]


## P1.3 Ethics (8 marks)

**(updated on 16/03/2022)**

Imagine you are **the head of a data mining company that needs to use** the insights gained in this assignment to scan social media for covid-related content, and automatically flag it as conspiracy or not conspiracy (for example, for hiding potentially harmful tweets or Facebook posts). **Some information about the project and the team:**

 - Your client is a political party concerned about misinformation.
 - The project requires mining Facebook, Reddit and Instagram data.
 - The team consists of Joe, an American mathematician who just finished college; Fei, a senior software engineer from China; and Francisco, a data scientist from Spain.

Reflect on the impact of exploiting data science for such an application. You should map your discussion to one of the five actions outlined in the UK’s Data Ethics Framework. 

Your answer should address the following:

 - Identify the action **in which your project is the weakest**.
 - Then, justify your choice by critically analyzing the three key principles **for that action** outlined in the Framework, namely transparency, accountability and fairness.
 - Finally, you should propose one solution that explicitly addresses one point related to one of these three principles, reflecting on how your solution would improve the data cycle in this particular use case.

Your answer should be between 500 and 700 words. **You are strongly encouraged to follow a scholarly approach, e.g., with references to peer reviewed publications. References do not count towards the word limit.**

---

COVID-related content is a very touchy subject to some, if not most people. Depending on the nationality, origin and/or political beliefs different people might react differently to the same content(Yoo, J., Dutra, S.V.O., Fanfan, D. et al., 2020). Luckily, our team consists of people with different backgrounds and ages. Diversity plays a big part in this project since different people with such different ethical backgrounds can analyze and sort information differently(Sylvia Ann Hewlett, Melinda Marshall, and Laura Sherbin, 2013). It helps a lot in identifying misuse of information or straight-up hate. The biggest problem, in my opinion, in identifying such content as potentially harmful is that it might offend some people even if an article or a post is written for educational reasons. In the modern world, people, in their masses, are getting offended more and more each coming day, even if statements are backed by the research and were not intended to insult or hurt, but rather to teach and educate. For me, this is where the line should be drawn. One just can’t please everyone. We have a whole world at our fingertips to search, analyze and think about the information we gather from different sources. Nobody is forcing anyone to read or react to everything they see. Nobody has cancelled freedom of expression, as long as there is no hate speech or a direct intent to repress. Transparency is key with information of this kind, since it may save lives, which is the most important thing we have. However, every content on this topic needs to be carefully reviewed, as, even if it’s properly cited, there is always a chance for misuse of information. Social media is a great platform for the freedom of speech, however, there has been little to no accountability, among regular people, to one's words (Harry Bruinius, 2018). Anyone can, basically, write anything they wish, whether it’s true or false, right or wrong, or offensive or harmless. We should always remember that we do not know who is sitting behind the device, from which those messages are sent. This is where Data Science comes into clutch. With the power of machines, we can sort out information faster than it reaches people, who might not be very pleased with the information provided. However, we should be fair to each publication, regardless of its origin. The only things which should matter are its intent, wording, conclusion and proper use of information. A particular solution to the misuse of information could be an implementation of responsibilities by the government. Although superficially existent, the government’s intervention in particular cases of the use of information would be of great help to society as a whole, since people need to only know the truth about the topics of such seriousness. Everyone must feel obligated to provide well-researched information, as well as a valid interpretation of it, in order to not mislead others, as it can cause severe damage to the readers as well as to lots of people around them. Such touchy topics like COVID, need to be highly monitored at all times, as it's in the government’s interest to convey reliable and truthful information to its citizens, permanent and temporary residents.

References

Yoo, J., Dutra, S.V.O., Fanfan, D. et al. Comparative analysis of COVID-19 guidelines from six countries: a qualitative study on the US, China, South Korea, the UK, Brazil, and Haiti. BMC Public Health 20, 1853 (2020). https://doi.org/10.1186/s12889-020-09924-7

Sylvia Ann Hewlett, Melinda Marshall, and Laura Sherbin. How Diversity Can Drive Innovation. Harvard Business Review (2013). https://hbr.org/2013/12/how-diversity-can-drive-innovation

Harry Bruinius. Are you what you post? Social media and the accountability debate. The Christian Science Monitor (2018). https://www.csmonitor.com/USA/Politics/2018/0809/Are-you-what-you-post-Social-media-and-the-accountability-debate

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/923108/Data_Ethics_Framework_2020.pdf


